<a href="https://colab.research.google.com/github/PosgradoMNA/actividades-del-projecto-equipo_36/blob/main/Reto_Aguas_subterraneas_Entrega2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Maestría en Inteligencia Artificial Aplicada**
##**Ciencia y Analítica de Datos**
###Tecnológico de Monterrey
###Prof Titular: **Dra. María de la Paz Rico**
###Tutor: **Roberto Antonio Guevara González**

####**Equipo 36**
####*Marcos Chávez - A01688507*
####*Bernanrdo Mijangos - A01793654*

https://github.com/PosgradoMNA/actividades-del-projecto-equipo_36/blob/main/Reto_Aguas_subterraneas_Entrega1.ipynb

####*Fecha de entrega 18/11/2022*

###**Base de datos Aguas Subterraneas**

In [257]:
from sklearn import linear_model as lm
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import scipy.stats as sts
import pandas as pd
import scipy as scp
import numpy as np
import sklearn.preprocessing
from sklearn.model_selection import train_test_split  ### for train and test split package
from sklearn import metrics  ## For calculation of MSE & RMSE
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, f1_score

import requests, zipfile
from io import BytesIO

from sklearn.cluster import KMeans
from tqdm import tqdm
from shapely.geometry import Point
from yellowbrick.cluster import SilhouetteVisualizer, KElbowVisualizer
from sklearn.metrics import silhouette_score
from geopy.geocoders import Nominatim
import geopy.distance

from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import matplotlib.colors

import sklearn.preprocessing as preprocessing
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RepeatedStratifiedKFold, cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, make_scorer
#from sklearn.model_selection import train_test_split, cross_validate,  RepeatedStratifiedKFold, learning_curve, validation_curve, GridSearch

from packaging import version
import sklearn


In [124]:
url = 'http://201.116.60.46/Datos_de_calidad_del_agua_de_5000_sitios_de_monitoreo.zip'

In [125]:
req = requests.get(url)
zipfile.ZipFile(BytesIO(req.content)).extractall('unzipped_zip/')
df=pd.read_csv('unzipped_zip/Datos_de_calidad_del_agua_2020/Datos_de_calidad_del_agua_de_sitios_de_monitoreo_de_aguas_subterraneas_2020.csv', encoding = 'latin1')
df.head()

,CLAVE,SITIO,ORGANISMO_DE_CUENCA,ESTADO,MUNICIPIO,ACUIFERO,SUBTIPO,LONGITUD,LATITUD,PERIODO,...,CUMPLE_CON_DUR,CUMPLE_CON_CF,CUMPLE_CON_NO3,CUMPLE_CON_AS,CUMPLE_CON_CD,CUMPLE_CON_CR,CUMPLE_CON_HG,CUMPLE_CON_PB,CUMPLE_CON_MN,CUMPLE_CON_FE
0,DLAGU6,POZO SAN GIL,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,ASIENTOS,VALLE DE CHICALOTE,POZO,-102.02210,22.20887,2020,...,SI,SI,SI,SI,SI,SI,SI,SI,SI,SI
1,DLAGU6516,POZO R013 CAÑADA HONDA,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,AGUASCALIENTES,VALLE DE CHICALOTE,POZO,-102.20075,21.99958,2020,...,SI,SI,SI,SI,SI,SI,SI,SI,SI,SI
2,DLAGU7,POZO COSIO,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,COSIO,VALLE DE AGUASCALIENTES,POZO,-102.28801,22.36685,2020,...,SI,SI,SI,NO,SI,SI,SI,SI,SI,SI
3,DLAGU9,POZO EL SALITRILLO,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,RINCON DE ROMOS,VALLE DE AGUASCALIENTES,POZO,-102.29449,22.18435,2020,...,SI,SI,SI,SI,SI,SI,SI,SI,SI,SI
4,DLBAJ107,RANCHO EL TECOLOTE,PENINSULA DE BAJA CALIFORNIA,BAJA CALIFORNIA SUR,LA PAZ,TODOS SANTOS,POZO,-110.24480,23.45138,2020,...,SI,SI,NO,SI,SI,SI,SI,SI,SI,SI


In [126]:
binarias = ['CUMPLE_CON_ALC', 'CUMPLE_CON_COND', 'CUMPLE_CON_SDT_ra',
            'CUMPLE_CON_SDT_salin', 'CUMPLE_CON_FLUO', 'CUMPLE_CON_DUR',
            'CUMPLE_CON_CF', 'CUMPLE_CON_NO3', 'CUMPLE_CON_AS', 'CUMPLE_CON_CD',
            'CUMPLE_CON_CR', 'CUMPLE_CON_HG', 'CUMPLE_CON_PB', 'CUMPLE_CON_MN',
            'CUMPLE_CON_FE']

###**Selecciona tus variables independientes X y dependiente Y (semáforo)**

In [127]:
# cambiamos los ND por SI

df.CUMPLE_CON_ALC = df.CUMPLE_CON_ALC.replace("ND","SI")
df.CUMPLE_CON_COND = df.CUMPLE_CON_COND.replace("ND","SI")
df.CUMPLE_CON_SDT_ra = df.CUMPLE_CON_SDT_ra.replace("ND","SI")
df.CUMPLE_CON_SDT_salin = df.CUMPLE_CON_SDT_salin.replace("ND","SI")
df.CUMPLE_CON_DUR = df.CUMPLE_CON_DUR.replace("ND","SI")
df.CUMPLE_CON_NO3 = df.CUMPLE_CON_NO3.replace("ND","SI")


In [129]:
X = df[binarias]
Y = df['SEMAFORO']

In [130]:

X.describe()

,CUMPLE_CON_ALC,CUMPLE_CON_COND,CUMPLE_CON_SDT_ra,CUMPLE_CON_SDT_salin,CUMPLE_CON_FLUO,CUMPLE_CON_DUR,CUMPLE_CON_CF,CUMPLE_CON_NO3,CUMPLE_CON_AS,CUMPLE_CON_CD,CUMPLE_CON_CR,CUMPLE_CON_HG,CUMPLE_CON_PB,CUMPLE_CON_MN,CUMPLE_CON_FE
count,1068,1068,1068,1068,1068,1068,1068,1068,1068,1068,1068,1068,1068,1068,1068
unique,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
top,SI,SI,SI,SI,SI,SI,SI,SI,SI,SI,SI,SI,SI,SI,SI
freq,1009,945,997,997,876,842,1007,986,941,1066,1053,1067,1056,982,932


In [134]:
X.columns

Index(['CUMPLE_CON_ALC', 'CUMPLE_CON_COND', 'CUMPLE_CON_SDT_ra',
       'CUMPLE_CON_SDT_salin', 'CUMPLE_CON_FLUO', 'CUMPLE_CON_DUR',
       'CUMPLE_CON_CF', 'CUMPLE_CON_NO3', 'CUMPLE_CON_AS', 'CUMPLE_CON_CD',
       'CUMPLE_CON_CR', 'CUMPLE_CON_HG', 'CUMPLE_CON_PB', 'CUMPLE_CON_MN',
       'CUMPLE_CON_FE'],
      dtype='object')

In [143]:
le = preprocessing.LabelEncoder()
X_t=pd.DataFrame()
for i in X.columns:
  le.fit(X[i])
  LabelEncoder()
  X_t[i]=le.transform(X[i])


In [194]:
#print(X_t.iloc[:,1],X.iloc[:,1])
for i in X_t.columns:
  print(X_t[i].value_counts())
  

1    1009
0      59
Name: CUMPLE_CON_ALC, dtype: int64
1    945
0    123
Name: CUMPLE_CON_COND, dtype: int64
1    997
0     71
Name: CUMPLE_CON_SDT_ra, dtype: int64
1    997
0     71
Name: CUMPLE_CON_SDT_salin, dtype: int64
1    876
0    192
Name: CUMPLE_CON_FLUO, dtype: int64
1    842
0    226
Name: CUMPLE_CON_DUR, dtype: int64
1    1007
0      61
Name: CUMPLE_CON_CF, dtype: int64
1    986
0     82
Name: CUMPLE_CON_NO3, dtype: int64
1    941
0    127
Name: CUMPLE_CON_AS, dtype: int64
1    1066
0       2
Name: CUMPLE_CON_CD, dtype: int64
1    1053
0      15
Name: CUMPLE_CON_CR, dtype: int64
1    1067
0       1
Name: CUMPLE_CON_HG, dtype: int64
1    1056
0      12
Name: CUMPLE_CON_PB, dtype: int64
1    982
0     86
Name: CUMPLE_CON_MN, dtype: int64
1    932
0    136
Name: CUMPLE_CON_FE, dtype: int64


###**Cambia a label encoding el semáforo, ej, de ["clase 1", "clase 2", "clase 3"] a [ 1,2,3]**

In [ ]:
le = preprocessing.LabelEncoder()

In [ ]:
le.fit(['Verde', 'Amarillo', 'Rojo'])

LabelEncoder()

In [ ]:
list(le.classes_)

['Amarillo', 'Rojo', 'Verde']

In [ ]:
le.transform(Y)

array([2, 2, 1, ..., 1, 2, 2])

In [ ]:
 Y_le = pd.DataFrame(le.transform(Y))
 Y_le.value_counts()

2    434
1    387
0    247
dtype: int64

In [ ]:
Y.value_counts()

Verde       434
Rojo        387
Amarillo    247
Name: SEMAFORO, dtype: int64

In [ ]:
le = preprocessing.LabelEncoder()


In [177]:
le = preprocessing.LabelEncoder()
le.fit(Y)
Y_t=pd.DataFrame(le.transform(Y))

In [219]:
Y_t.value_counts()

1    434
0    387
2    247
dtype: int64

In [218]:
Y.value_counts()

Verde       434
Rojo        387
Amarillo    247
Name: SEMAFORO, dtype: int64

In [226]:
Y_t[0] = Y_t[0].replace([2,1,0],[2,1,3])

In [227]:
Y_t.value_counts()

1    434
3    387
2    247
dtype: int64

###**Realiza un análisis general de las features importances a traves de decision trees o random forest**

In [228]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, random_state=42)
rnd_clf.fit(X_t, Y_t)
for score, name in zip(rnd_clf.feature_importances_, X_t.columns):
    print(round(score, 2), name)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


0.03 CUMPLE_CON_ALC
0.04 CUMPLE_CON_COND
0.01 CUMPLE_CON_SDT_ra
0.01 CUMPLE_CON_SDT_salin
0.23 CUMPLE_CON_FLUO
0.17 CUMPLE_CON_DUR
0.09 CUMPLE_CON_CF
0.12 CUMPLE_CON_NO3
0.12 CUMPLE_CON_AS
0.0 CUMPLE_CON_CD
0.03 CUMPLE_CON_CR
0.0 CUMPLE_CON_HG
0.02 CUMPLE_CON_PB
0.07 CUMPLE_CON_MN
0.07 CUMPLE_CON_FE


###**Selecciona las variables de mayor importancia**

In [230]:
X_var = X_t[['CUMPLE_CON_FLUO','CUMPLE_CON_DUR','CUMPLE_CON_NO3','CUMPLE_CON_AS','CUMPLE_CON_CF']].copy()

In [232]:
X_var.describe()

,CUMPLE_CON_FLUO,CUMPLE_CON_DUR,CUMPLE_CON_NO3,CUMPLE_CON_AS,CUMPLE_CON_CF
count,1068.000000,1068.000000,1068.000000,1068.000000,1068.000000
mean,0.820225,0.788390,0.923221,0.881086,0.942884
std,0.384180,0.408641,0.266365,0.323839,0.232173
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,1.000000,1.000000,1.000000
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000


###**Realiza tu clasificador, recuerda dividir los datos de manera balanceada (auxiliate de train test split)**

In [234]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_var, Y_t, test_size = 0.2, random_state = 35)

In [245]:
modelos = [DecisionTreeClassifier(), RandomForestClassifier()]
nombres_modelos = ['decision_tree', 'random_forest']

In [265]:
#  metricas y funciones
# Metric functions

def recall(yreal, ypred):
    cm = confusion_matrix(yreal, ypred).flatten()
    vp = cm[3]
    fn = cm[2]
  
    recall = vp/(vp+fn)
    return recall

def gmean(yreal, ypred):
    cm = confusion_matrix(yreal, ypred).flatten()
    recall = cm[3]/(cm[2]+cm[3])
    especificidad = cm[0]/(cm[1]+cm[0])
     
    gmean = np.sqrt(recall*especificidad)

    return gmean

def accuracy(yreal, ypred):
    
    cm = confusion_matrix(yreal, ypred).flatten()
    vp = cm[3]
    vn = cm[0]
    fp = cm[1]
    fn = cm[2]
    
    accuracy = (vp+vn)/(vp+vn+fn+fp)
    return accuracy

def precision(yreal, ypred):
    
    cm = confusion_matrix(yreal, ypred).flatten()
    vp = cm[3]
    fp = cm[1]
    
    precision = vp/(vp+fp)
    return precision

def f1_score(yreal, ypred):
    
    recall_metric = recall(yreal, ypred)
    precision_metric = precision(yreal, ypred)
    
    f1 = 2 * (recall_metric*precision_metric)/(recall_metric+precision_metric)
    return f1

def mi_cm(yreal, ypred):
  cm = confusion_matrix(yreal, ypred)
  
  texto = ['Verdaderos negativos', 'Falsos positivos', 'Falsos negativos', 'Verdaderos positivos']
  siglas = ['(VN)', '(FP)', '(FN)', 'VP']
  valores = cm.flatten().tolist()
  porcentaje = [f'{np.round(value, 1)}%' for value in (cm.flatten()/cm.flatten().sum()) * 100]
  
  labels = np.array([f"{v1}\n{v2}\n{v3}\n{v4}" for v1, v2, v3, v4 in zip(texto, siglas, valores ,porcentaje)]).reshape(2, 2)

  fig, axs = plt.subplots(figsize=(6,4))
  sns.heatmap(cm, annot=labels, fmt='', cmap='Spectral', ax=axs, cbar=False)
  axs.set_xlabel('Etiquetas de predicción')
  axs.set_ylabel('Etiquetas de Reales')
  plt.show()


In [266]:
kfold = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=0) 

In [267]:
resultados = list()

for i in range(len(modelos)):
  
  metricas = {'accuracy':make_scorer(accuracy), 
                 'precision': make_scorer(precision), 
                 'recall': make_scorer(recall),
                 'f1_score': make_scorer(f1_score),
                 'gmean': make_scorer(gmean)
              }
  
#  X_data_t = col_transform.fit_transform(X)
  
  # X_data = col_transform.fit_transform(X)
  resultadosOU = cross_validate(modelos[i], X_train, np.ravel(y_train), scoring=metricas, cv=kfold)
  resultados.append(resultadosOU)
  print(nombres_modelos[i].upper())
  print('Accuracy:', round((np.mean(resultadosOU['test_accuracy'])),3))
  print('Precision:', round((np.mean(resultadosOU['test_precision'])),3))
  print('Recall:', round((np.mean(resultadosOU['test_recall'])),3))
  print('F1-score:', round((np.mean(resultadosOU['test_f1_score'])),3))
  print('gmean:', round((np.mean(resultadosOU['test_gmean'])),3))
  print('\n')


DECISION_TREE
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-score: 1.0
gmean: 1.0


RANDOM_FOREST
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-score: 1.0
gmean: 1.0




In [268]:
resultados = list()

for i in range(len(modelos)):
  
  metricas = {'accuracy':make_scorer(accuracy), 
                 'precision': make_scorer(precision), 
                 'recall': make_scorer(recall),
                 'f1_score': make_scorer(f1_score),
                 'gmean': make_scorer(gmean)
              }
  
  X_data_t = col_transform.fit(X_train)
  X_transform = X_data_t.transform(X_train)
  
  # X_data = col_transform.fit_transform(X)
  resultadosOU = cross_validate(modelos[i], X_transform, np.ravel(y_train), scoring=metricas, cv=kfold)
  resultados.append(resultadosOU)
  print(nombres_modelos[i].upper())
  print('Accuracy:', round((np.mean(resultadosOU['test_accuracy'])),3))
  print('Precision:', round((np.mean(resultadosOU['test_precision'])),3))
  print('Recall:', round((np.mean(resultadosOU['test_recall'])),3))
  print('F1-score:', round((np.mean(resultadosOU['test_f1_score'])),3))
  print('gmean:', round((np.mean(resultadosOU['test_gmean'])),3))
  print('\n')


NameError: ignored

In [258]:
# Random Forests

from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16,
                                 n_jobs=-1, random_state=42)
rnd_clf.fit(X_train, y_train)
y_pred_rf = rnd_clf.predict(X_test)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


In [259]:
y_pred_rf

array([3, 3, 1, 3, 1, 3, 1, 3, 3, 1, 1, 3, 1, 1, 3, 3, 2, 1, 3, 1, 1, 2,
       2, 2, 1, 1, 1, 2, 3, 2, 3, 1, 2, 3, 3, 2, 1, 3, 3, 3, 3, 1, 1, 1,
       3, 3, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 3, 1, 3, 2, 1, 2, 1, 1, 1, 1,
       1, 3, 3, 3, 3, 3, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1,
       1, 2, 3, 3, 3, 3, 2, 2, 3, 1, 1, 3, 1, 1, 1, 3, 1, 1, 3, 3, 3, 2,
       3, 1, 1, 3, 3, 3, 1, 3, 1, 2, 1, 1, 2, 3, 1, 1, 1, 3, 3, 3, 3, 3,
       1, 3, 3, 1, 3, 1, 3, 2, 3, 2, 1, 1, 1, 3, 1, 2, 1, 1, 1, 2, 3, 2,
       3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 3, 3, 2, 1, 1, 2, 2, 2, 1,
       1, 3, 3, 1, 2, 1, 1, 3, 1, 1, 1, 1, 1, 3, 3, 1, 1, 3, 3, 3, 1, 1,
       3, 1, 1, 1, 3, 3, 2, 2, 1, 2, 2, 1, 3, 1, 1, 3])

In [263]:
# Desicion Tree

tree_clf = DecisionTreeClassifier(max_depth=3, random_state=42)
tree_clf.fit(X_train, y_train)

y_pred = tree_clf.predict(X_test)

In [264]:
y_pred

array([3, 3, 1, 3, 1, 3, 1, 1, 3, 1, 1, 3, 1, 1, 3, 3, 2, 1, 1, 1, 1, 2,
       2, 2, 1, 1, 1, 2, 1, 2, 3, 1, 2, 3, 3, 2, 1, 3, 3, 3, 3, 1, 1, 1,
       1, 3, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1,
       1, 3, 3, 3, 3, 2, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1,
       1, 2, 1, 2, 1, 1, 2, 2, 3, 1, 1, 3, 1, 1, 1, 2, 1, 1, 3, 1, 3, 2,
       3, 1, 1, 3, 2, 1, 1, 1, 1, 2, 1, 1, 2, 3, 1, 1, 1, 3, 3, 3, 3, 1,
       1, 3, 3, 1, 3, 1, 2, 2, 3, 2, 1, 1, 1, 3, 1, 2, 1, 1, 1, 2, 3, 2,
       3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 3, 3, 2, 1, 1, 2, 2, 2, 1,
       1, 1, 3, 1, 2, 1, 1, 3, 1, 1, 1, 1, 1, 3, 3, 1, 1, 1, 3, 3, 1, 1,
       3, 1, 1, 1, 3, 1, 2, 2, 1, 2, 2, 1, 1, 1, 1, 3])

###**Explora que clasificador es el más optimo, ejemplo:**
###-*Decision trees*
###-*Random Forest*

###**Determina el grado de exactitud a través del reporte de clasificación https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html**

###**Visualiza los resultados del modelo o las predicciones a través de una matriz de confusión**